**Hyperparameters**

In [ ]:
import math
import random
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.models import Sequential
from keras.preprocessing import image
from keras.applications.imagenet_utils import preprocess_input
from os import path
from PIL import Image
import matplotlib.pyplot as plt
import keras.backend as K
from sklearn.model_selection import train_test_split

In [ ]:
epochs = 30
batch_size = 4
margin = 1   # Margin for constrastive loss.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
base_dataset_path = "/content/drive/MyDrive/Sizvy_Towhid_Rigan/"
dataset_version = "Final_Database/"

roi_folder_name = 'yolo_vein_roi'
# roi_folder_name = 'ROI'

number_of_roi_per_user = 5

landmarks = ["dorsal", "wrist"]

# ! ls /content/drive/MyDrive/vein_dataset/data_v1/

In [ ]:
import os
import pandas as pd

base_path = base_dataset_path + dataset_version
print(base_path)
users = next(os.walk(base_path))[1]
#sort users
users.sort()

# load a csv file
df = pd.read_csv("/content/data_quality.csv")


very_good_difference = {"dorsal" : [], "wrist" : []}
good_difference = {"dorsal" : [], "wrist" : []}
average_difference = {"dorsal" : [], "wrist" : []}
swapped_photo = {"dorsal": [], "wrist" : []}
no_difference = {"dorsal": [], "wrist" : []}
data_bad = {"dorsal": [], "wrist" : []}


for index, row in df.iterrows():
    if "a" == row["dorsal"]:
        very_good_difference["dorsal"].append(row['x'])
    elif "b" == row['dorsal']:
        good_difference["dorsal"].append(row['x'])
    elif "c" == row["dorsal"]:
        average_difference["dorsal"].append(row['x'])
    elif "d" == row["dorsal"]:
        swapped_photo["dorsal"].append(row['x'])
    elif "e" == row["dorsal"]:
        no_difference["dorsal"].append(row['x'])
    elif "f" == row["dorsal"]:
        data_bad["dorsal"].append(row['x'])
    else:
        print(row['x'])

    if "a" == row["wrist"]:
        very_good_difference["wrist"].append(row['x'])
    elif "b" == row['wrist']:
        good_difference["wrist"].append(row['x'])
    elif "c" == row["wrist"]:
        average_difference["wrist"].append(row['x'])
    elif "d" == row["wrist"]:
        swapped_photo["wrist"].append(row['x'])
    elif "e" == row["wrist"]:
        no_difference["wrist"].append(row['x'])
    elif "f" == row["wrist"]:
        data_bad["wrist"].append(row['x'])


print("Dorsal")
print("Very good cnt: ", len(very_good_difference["dorsal"]))
print("Good cnt: ", len(good_difference["dorsal"]))
print("Average cnt: ", len(average_difference["dorsal"]))
print("Swapped cnt : ", len(swapped_photo["dorsal"]))
print("No difference cnt: ", len(no_difference["dorsal"]))
print("Data bad cnt: ", len(data_bad["dorsal"]))


total = len(very_good_difference["dorsal"]) + len(good_difference["dorsal"]) + len(average_difference["dorsal"]) + len(swapped_photo["dorsal"]) +  len(no_difference["dorsal"]) + len(data_bad["dorsal"])

print("Dorsal total : ", total)


print("Wrist")
print("Very good cnt: ", len(very_good_difference["wrist"]))
print("Good cnt: ", len(good_difference["wrist"]))
print("Average cnt: ", len(average_difference["wrist"]))
print("Swapped cnt : ", len(swapped_photo["wrist"]))
print("No difference cnt: ", len(no_difference["wrist"]))
print("Data bad cnt: ", len(data_bad["wrist"]))

total = len(very_good_difference["wrist"]) + len(good_difference["wrist"]) + len(average_difference["wrist"]) + len(swapped_photo["wrist"]) +  len(no_difference["wrist"]) + len(data_bad["wrist"])

print("Wrist total : ", total)

dorsal_good = very_good_difference["dorsal"] + good_difference["dorsal"] + swapped_photo["dorsal"]
dorsal_average = average_difference["dorsal"]
dorsal_good_average = dorsal_good + dorsal_average

wrist_good = very_good_difference["wrist"] + good_difference["wrist"] + swapped_photo["wrist"]
wrist_average = average_difference["wrist"]
wrist_good_average = wrist_good + wrist_average

# take all the intersection of dorsal_good_average and wrist_good_average
#intersection = list(set(dorsal_good) & set(wrist_good))
#intersection = dorsal_good
intersection = []
for name in users:
  if name.endswith("_band"):
    intersection.append(name)
# make intersection a set
intersection = list(set(intersection))
print(intersection)

intersection = wrist_good

remove_users = ["sizvy", "mansur", "hasina"]
# remove  remove_users from intersection list if the user is present
for user in remove_users:
    if user in intersection:
        intersection.remove(user)

print("Intersection after delete: ", len(intersection))

rantom_persons = random.sample(very_good_difference["dorsal"], 5)

#intersection = intersection + rantom_persons
# shuffle the intersection list
#intersection = random.sample(intersection, 25)
intersection = random.sample(intersection, 40)


print(intersection)
print("Intersection: ", len(intersection))






users = intersection
length = len(users)
print(length)
training_len = (int)(length*0.9)
users_train_val = users[:training_len]
users_test = users[training_len:]

print(users_train_val)
print(users_test)

/content/drive/MyDrive/Sizvy_Towhid_Rigan/Final_Database/
Dorsal
Very good cnt:  20
Good cnt:  25
Average cnt:  17
Swapped cnt :  19
No difference cnt:  4
Data bad cnt:  0
Dorsal total :  85
Wrist
Very good cnt:  11
Good cnt:  28
Average cnt:  19
Swapped cnt :  8
No difference cnt:  17
Data bad cnt:  2
Wrist total :  85
['dear_band', 'huda_band', 'abdullah_band', 'nakib_band', 'tamim_17_band', 'shakib_emn_band', 'sadat_band', 'emon_17_band', 'shakib_17_band', 'alamin_17_band', 'mosto_sharif_band', 'mehrab_band', 'rakib_band', 'sharif_17_band', 'asif18_band', 'nurul_band', 'tm_band', 'sakib_emn2_band', 'zakir_band', 'pappu_band']
Intersection after delete:  44
['per2', 'mujahid', 'shahan', 'per14', 'per1', 'tushar_5003', 'per84', 'rigan', 'per13', 'tm_band', 'per8', 'zahid', 'emon_5011', 'tanvir_5001', 'rakib_band', 'emon_5001', 'farsi1', 'fazle_rabbi_5011', 'towhid', 'shakib_emn_band', 'per5', 'dipto_17', 'nafiz', 'rabbi_5009', 'mahir_5003', 'tamim_17_band', 'sadat_band', 'shirin', 'pe

In [ ]:
# import os
def list_files_in_directory(directory):
  files = []
  for file in os.listdir(directory):
    # Join the directory path with the file name to get the full path
    full_path = os.path.join(directory, file)
    if os.path.isfile(full_path):
      files.append(full_path)
  return files

def get_file_path_list(dir):
  # Get a list of all files in the current directory
  all_files = list_files_in_directory(dir)
  if len(all_files) < number_of_roi_per_user:
    first_file = all_files[0]
    while(len(all_files) < number_of_roi_per_user):
      all_files.append(first_file)
  # Get a list of all file paths in the current directory
  all_file_paths = [os.path.join(dir, file) for file in all_files]


  return all_file_paths[:number_of_roi_per_user]

In [ ]:
def create_triplets(landmark, hand, users):
  triplets = pd.DataFrame(columns=['anchor_image', 'hydrated_image', 'dehydrated_image'])

  index = 0

  for user in users:
    hydrated_list_url = base_dataset_path + dataset_version + user + "/" + "hydrated/" + hand + "/" + landmark + "/" + roi_folder_name
    hydrated_list = get_file_path_list(hydrated_list_url)

    #print("user: ", user)
    #print("hydrated: ", len(hydrated_list))

    dehydrated_list_url = base_dataset_path + dataset_version + user + "/" + "dehydrated/" + hand + "/" + landmark + "/" + roi_folder_name
    dehydrated_list = get_file_path_list(dehydrated_list_url)

    #print("dehydrated: ", len(dehydrated_list))
    if ((len(hydrated_list) == 0) or (len(dehydrated_list) == 0)):
      continue

    anchor = hydrated_list[0]

    for i in range(len(dehydrated_list)):
      dehydrated = dehydrated_list[i]
      for j in range(len(hydrated_list)):
        hydrated = hydrated_list[j]
        triplets.loc[len(triplets)] = [anchor, hydrated, dehydrated]

    #print("length of triplets: ", len(triplets))
  return triplets

In [ ]:
def load_images(triplets):
  anchor_images_array = []
  hydrated_images_array = []
  dehydrated_images_array = []

  errorIndices = []
  count = 0
  fileError = 0

  for i, directory in triplets['anchor_image'].items():
    try:
      img = image.load_img(directory, target_size=(200, 200))
      img = image.img_to_array(img)
      img = tf.image.convert_image_dtype(img, tf.float32)
      img = preprocess_input(img)

      anchor_images_array.append(img)
      count = count + 1
    except FileNotFoundError:
      fileError = fileError + 1

  print("anchor image array: ", len(anchor_images_array))

  for i, directory in triplets['hydrated_image'].items():
    try:
      img = image.load_img(directory, target_size=(200, 200))
      img = image.img_to_array(img)
      img = tf.image.convert_image_dtype(img, tf.float32)
      img = preprocess_input(img)

      hydrated_images_array.append(img)
      count = count + 1
    except FileNotFoundError:
      fileError = fileError + 1

  print("hydrated image array: ", len(hydrated_images_array))

  for i, directory in triplets['dehydrated_image'].items():
    try:
      img = image.load_img(directory, target_size=(200, 200))
      img = image.img_to_array(img)
      img = tf.image.convert_image_dtype(img, tf.float32)
      img = preprocess_input(img)

      dehydrated_images_array.append(img)
      count = count + 1
    except FileNotFoundError:
      fileError = fileError + 1

  print("dehydrated image array : ", len(dehydrated_images_array))

  image_triplets = []

  for i in range(len(anchor_images_array)):
    anchor_img = anchor_images_array[i]
    hydrated_img = hydrated_images_array[i]
    dehydrated_img = dehydrated_images_array[i]
    image_triplets += [[anchor_img, hydrated_img, dehydrated_img]]

  print("image triplets : ", len(image_triplets))
  return image_triplets

In [ ]:
def make_pairs(triplets):
    pairs = []
    labels = []

    for idx in range(len(triplets)):
      # add a matching example
      x1 = triplets[idx][0]
      x2 = triplets[idx][1]
      pairs += [[x1, x2]]
      labels += [1]

      # add a non-matching example
      x1 = triplets[idx][0]
      x2 = triplets[idx][2]
      pairs += [[x1, x2]]
      labels += [0]

    return np.array(pairs), np.array(labels).astype("float32")

In [ ]:
def split_dataset_pair(x, y):
  x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.2, random_state=42)

  # index 0 has anchor that means x_train_1 has acnhor images only and x_train_2 hydrated or dehydarted
  x_train_1 = x_train[:, 0]
  x_train_2 = x_train[:, 1]

  x_val_1 = x_val[:, 0]
  x_val_2 = x_val[:, 1]

  return (x_train_1, x_train_2, x_val_1, x_val_2, y_train, y_val)

In [ ]:
def euclidean_distance(vects):
    x, y = vects
    sum_square = tf.math.reduce_sum(tf.math.square(x - y), axis=1, keepdims=True)
    return tf.math.sqrt(tf.math.maximum(sum_square, tf.keras.backend.epsilon()))

def manhattan_distance(vects):
    x, y = vects
    return K.sum(K.abs(x - y), axis=1, keepdims=True)

def cosine_distance(vects):
    x, y = vects
    x_norm = tf.nn.l2_normalize(x, axis = 1)
    y_norm = tf.nn.l2_normalize(y, axis = 1)
    cos = tf.math.reduce_sum(x_norm * y_norm, axis = 1, keepdims=True)
    cosx = tf.clip_by_value(cos, -1.0, 1.0)
    return tf.math.abs(cos)

**Multi Modal Siamese Model**

Embedding for dorsal

In [ ]:
input = layers.Input((200, 200, 3))
x = tf.keras.layers.BatchNormalization()(input)
x = layers.Conv2D(4, (5, 5), activation="relu")(x)
x = layers.AveragePooling2D(pool_size=(2, 2))(x)
x = layers.Conv2D(16, (5, 5), activation="relu")(x)
x = layers.AveragePooling2D(pool_size=(2, 2))(x)
x = layers.Flatten()(x)

x = tf.keras.layers.BatchNormalization()(x)
x = layers.Dense(10, activation="relu")(x)
embedding_network_dorsal_left = keras.Model(input, x)
embedding_network_dorsal_right = keras.Model(input, x)

In [ ]:
input2 = layers.Input((200, 200, 3))
x2 = tf.keras.layers.BatchNormalization()(input2)
x2 = layers.Conv2D(5, (5, 5), activation="relu")(x2)
x2 = layers.AveragePooling2D(pool_size=(2, 2))(x2)
x2 = layers.Conv2D(20, (5, 5), activation="relu")(x2)
x2 = layers.AveragePooling2D(pool_size=(2, 2))(x2)
x2 = layers.Flatten()(x2)

x2 = tf.keras.layers.BatchNormalization()(x2)
x2 = layers.Dense(10, activation="relu")(x2)
embedding_network_dorsal_right = keras.Model(input2, x2)

Embedding for wrist

In [ ]:
# input = layers.Input((200, 200, 3))
# x = tf.keras.layers.BatchNormalization()(input)
# x = layers.Conv2D(5, (5, 5), activation="relu")(x)
# x = layers.AveragePooling2D(pool_size=(2, 2))(x)
# x = layers.Conv2D(25, (5, 5), activation="relu")(x)
# x = layers.AveragePooling2D(pool_size=(2, 2))(x)
# x = layers.Flatten()(x)

# x = tf.keras.layers.BatchNormalization()(x)
# x = layers.Dense(10, activation="relu")(x)
# embedding_network_wrist = keras.Model(input, x)

# Siamese model only for dorsal
*(Left hand + Right hand)* <br>
ref = anchor (hydrated) <br>
state = hydrated or dehydrated

left hand dorsal from both tower (ref and state tower) share same embedding means they share weight. <br>
same is for right hand dorsal

In [ ]:
input_left_dorsal_ref = layers.Input((200, 200, 3))
input_left_dorsal_state = layers.Input((200, 200, 3))

input_right_dorsal_ref = layers.Input((200, 200, 3))
input_right_dorsal_state = layers.Input((200, 200, 3))

tower_ref_left = embedding_network_dorsal_left(input_left_dorsal_ref)
tower_state_left = embedding_network_dorsal_left(input_left_dorsal_state)

tower_ref_right = embedding_network_dorsal_right(input_right_dorsal_ref)
tower_state_right = embedding_network_dorsal_right(input_right_dorsal_state)

# tower_ref feature merge
# number row must be same as before but feature will be merged
tower_ref = layers.Concatenate(axis=-1)([tower_ref_left, tower_ref_right])
tower_state = layers.Concatenate(axis=-1)([tower_state_left, tower_state_right])

merge_layer = layers.Lambda(euclidean_distance)([tower_ref, tower_state])
normal_layer = tf.keras.layers.BatchNormalization()(merge_layer)
output_layer = layers.Dense(1, activation="sigmoid")(normal_layer)

# order in inputs must be maintained
siamese = keras.Model(inputs=[input_left_dorsal_ref, input_right_dorsal_ref, input_left_dorsal_state, input_right_dorsal_state], outputs=output_layer)

In [ ]:
def loss(margin=1):
    def contrastive_loss(y_true, y_pred):
        square_pred = tf.math.square(y_pred)
        margin_square = tf.math.square(tf.math.maximum(margin - (y_pred), 0))
        return tf.math.reduce_mean(
            (1 - y_true) * square_pred + (y_true) * margin_square
        )

    return contrastive_loss


In [ ]:
siamese.compile(loss=loss(margin=margin), optimizer="RMSprop", metrics=["accuracy"])
siamese.summary()

Model: "model_19"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_27 (InputLayer)       [(None, 200, 200, 3)]        0         []                            
                                                                                                  
 input_29 (InputLayer)       [(None, 200, 200, 3)]        0         []                            
                                                                                                  
 input_28 (InputLayer)       [(None, 200, 200, 3)]        0         []                            
                                                                                                  
 input_30 (InputLayer)       [(None, 200, 200, 3)]        0         []                            
                                                                                           

In [ ]:
#load train data
left_hand = "left_hand"
right_hand = "right_hand"
landmark = "wrist"

triplets_left_hand = create_triplets(landmark, left_hand, users_train_val)
triplets_left_hand = triplets_left_hand[triplets_left_hand.notna()]

triplets_right_hand = create_triplets(landmark, right_hand, users_train_val)
triplets_right_hand = triplets_right_hand[triplets_right_hand.notna()]

image_triplets_left_hand = load_images(triplets_left_hand)
image_triplets_right_hand = load_images(triplets_right_hand)

res_left_hand = make_pairs(image_triplets_left_hand)
res_right_hand = make_pairs(image_triplets_right_hand)

# res has pairs and labels
# pairs[index] = [x1, x2]
# where x1 = anchor and x2 = hydrated or dehydrated
# labels[index] = 0 if x2 dehydrated 1 if hydrated
pairs_left_hand = res_left_hand[0]
labels_left_hand = res_left_hand[1]

pairs_right_hand = res_right_hand[0]
labels_right_hand = res_right_hand[1]

# indx 0, 2, 4, ... all even has hydrated for both hand
# indx 1, 3, 5, ... all odd has dehydrated for both hand
print("check label left right disim: ", len(labels_left_hand!=labels_right_hand))

(x_train_left_dorsal_ref, x_train_left_dorsal_state, x_val_left_dorsal_ref,
x_val_left_dorsal_state, y_train_left_dorsal, y_val_left_dorsal) = split_dataset_pair(pairs_left_hand, labels_left_hand)

(x_train_right_dorsal_ref, x_train_right_dorsal_state, x_val_right_dorsal_ref,
x_val_right_dorsal_state, y_train_right_dorsal, y_val_right_dorsal) = split_dataset_pair(pairs_right_hand, labels_right_hand)

anchor image array:  900
hydrated image array:  900
dehydrated image array :  900
image triplets :  900
anchor image array:  900
hydrated image array:  900
dehydrated image array :  900
image triplets :  900
check label left right disim:  1800


In [ ]:
# load test data
triplets_left_hand_test = create_triplets(landmark, left_hand, users_test)
triplets_left_hand_test = triplets_left_hand_test[triplets_left_hand_test.notna()]

triplets_right_hand_test = create_triplets(landmark, right_hand, users_test)
triplets_right_hand_test = triplets_right_hand_test[triplets_right_hand_test.notna()]

image_triplets_left_hand_test = load_images(triplets_left_hand_test)
image_triplets_right_hand_test = load_images(triplets_right_hand_test)

res_left_hand_test = make_pairs(image_triplets_left_hand_test)
res_right_hand_test = make_pairs(image_triplets_right_hand_test)

pairs_left_hand_test = res_left_hand_test[0]
labels_left_hand_test = res_left_hand_test[1]

pairs_right_hand_test = res_right_hand_test[0]
labels_right_hand_test = res_right_hand_test[1]

x_test_left_dorsal_ref = pairs_left_hand_test[:, 0]
x_test_left_dorsal_state = pairs_left_hand_test[:, 1]

x_test_right_dorsal_ref = pairs_right_hand_test[:, 0]
x_test_right_dorsal_state = pairs_right_hand_test[:, 1]

In [ ]:
def landmark_model(landmark):
  history = siamese.fit(
    [x_train_left_dorsal_ref, x_train_right_dorsal_ref, x_train_left_dorsal_state, x_train_right_dorsal_state],
    y_train_left_dorsal,
    validation_data=([x_val_left_dorsal_ref, x_val_right_dorsal_ref, x_val_left_dorsal_state, x_val_right_dorsal_state],
                     y_val_left_dorsal),
    batch_size=batch_size,
    epochs=epochs,
  )

  predictions = siamese.predict([x_test_left_dorsal_ref, x_test_right_dorsal_ref, x_test_left_dorsal_state, x_test_right_dorsal_state])

  return history, predictions, labels_left_hand_test


In [ ]:
history_dorsal, predictions_dorsal, y_test_dorsal = landmark_model('wrist')

# **Visualize**

In [ ]:
def plt_metric(history, metric, title, has_valid=True):
    plt.plot(history[metric])
    if has_valid:
        plt.plot(history["val_" + metric])
        plt.legend(["train", "validation"], loc="upper left")
    plt.title(title)
    plt.ylabel(metric)
    plt.xlabel("epoch")
    plt.show()

In [ ]:
# Plot the accuracy
plt_metric(history=history_dorsal.history, metric="accuracy", title="Model accuracy")

# Plot the constrastive loss
plt_metric(history=history_dorsal.history, metric="loss", title="Constrastive Loss")

In [ ]:
# calculate test accuracy
test_accuracy = np.sum(np.round(predictions_dorsal) == y_test_dorsal) / len(y_test_dorsal)
print("Test accuracy:", test_accuracy)

In [ ]:
correct = 0
total = 0
for i in range(len(predictions_dorsal)):
  if predictions_dorsal[i] >= 0.5:
    predictions_dorsal[i] = 1
  else:
    predictions_dorsal[i] = 0
  if predictions_dorsal[i] == y_test_dorsal[i]:
    correct += 1
  total += 1

print("Accuracy: ", correct/total)